Wedge Part 2

In [19]:
import os
import re
import datetime 
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_gbq
import janitor
from random import sample
# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

from os.path import exists


Establishing a connection with GBQ

In [20]:
# These first two values will be different on your machine. 
service_path = "/Users/willd/Desktop/ADA/"
service_file = 'msba-wd-7a44c8ddeff2.json' # change this to your authentication information  
gbq_proj_id = 'msba-wd' # 'msba-wd' # change this to your project. 

# And this should stay the same. 
private_key =service_path + service_file

print(exists(private_key))

# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

print(credentials)

# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

for item in client.list_datasets() : 
    print(item.full_dataset_id)

True
msba-wd:class_example
msba-wd:dram_shop
msba-wd:wedge_proj


Writing a query to GBQ and printing into a pandas dataframe.
This task asks you to generate a file of owners where the file contains every record for each owner. There will be more than one owner in the file, and I do not want you to include card_no==3, which is the code for non-owners.

In [39]:
query = """
SELECT distinct(card_no)
FROM `msba-wd.wedge_proj.transArchive*`
Where card_no != 3
"""

owner_data = pandas_gbq.read_gbq(query1,project_id = gbq_proj_id)
owner_data.head()



Downloading: 100%|█████████| 26997/26997 [00:00<00:00, 33064.62rows/s]


,card_no
0,45087
1,47885
2,48337
3,45052
4,48368


In [40]:
owner_list = []
results = client.query(query)


for row in results :
    owner_list.append(row.values()[0])
    
random.seed(10)
sample_owners = random.choices(owner_list, k=400)

In [55]:
query = """
SELECT * except(display,memtype)
FROM `msba-wd.wedge_proj.transArchive*`
Where card_no IN ( 
"""
query = query +",".join([str(element) for element in sample_owners])+")"


In [56]:
owner_data = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)
owner_data.head()

Downloading: 100%|████████| 888173/888173 [04:39<00:00, 3173.79rows/s]


,datetime,register_no,emp_no,trans_no,upc,description,trans_type,trans_subtype,trans_status,department,...,varflag,batchheaderid,local,organic,receipt,card_no,store,branch,match_id,trans_id
0,2013-01-02 18:26:42,4,74,101,0,Credit Card,T,CC,,0,...,0.0,NaN,0.0,NaN,0.0,40419,1,0,0.0,9
1,2013-01-03 19:39:33,7,14,66,0,Credit Card,T,CC,,0,...,0.0,NaN,0.0,NaN,0.0,44939,1,0,0.0,28
2,2013-01-04 11:53:37,16,54,98,0,Credit Card,T,CC,,0,...,0.0,NaN,0.0,NaN,0.0,47800,1,0,0.0,14
3,2013-01-04 14:14:23,6,64,31,0,Credit Card,T,CC,,0,...,0.0,NaN,0.0,NaN,0.0,48346,1,0,0.0,39
4,2013-01-05 13:20:19,16,54,127,0,Credit Card,T,CC,,0,...,0.0,NaN,0.0,NaN,0.0,47800,1,0,0.0,8
